In [54]:
import pandas as pd
import spacy
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import numpy as np
import requests
import nltk
nlp = spacy.load('en_core_web_sm')

import gpt_2_simple as gpt2

In [55]:
#Read in template questions,answers and memory
template_q = pd.read_csv('data/likes_question_templates.csv')
retrieval_q = pd.read_csv('data/questions_templates.csv')
template_a = pd.read_csv('data/answer_templates.csv')
retrieval_a = pd.read_csv('data/answer_templates_2.csv')
like_memory = pd.read_csv('data/sentiment_memory.csv')

In [56]:
#Some fruits are listed under "Food" topic, so the line below is temporary remove solution
like_memory = like_memory.drop_duplicates(subset='subject', keep="last")
#Assign random sentiment to every noun item in memory
temp = np.random.random(len(like_memory))
like_memory['sentiment'] = temp

In [57]:
#def calculate_topic_sent(): 
#Calculate topic average sentiment (not very useful considering random function mean 0.5)
topic_sent = {}
memory_topics = set(like_memory.topic)
for topic in memory_topics:
    topic_list = like_memory.loc[like_memory['topic'] == topic]
    count = 0
    divisor = len(topic_list)
    for i in range(divisor):
        count = count + topic_list.sentiment.iloc[i]
    topic_sent[topic] = count/divisor
print(topic_sent)

{'animal': 0.5266773475979342, 'food': 0.638869731751168, 'hobby': 0.4588529456011475, 'TV show': 0.5372296807329312, 'fruit': 0.49724013777262815, 'music': 0.36348344566163315, 'dessert': 0.4949504716896939, 'sport': 0.43641777358113737, 'genre': 0.31708426163962333, 'interest': 0.3105392522365218, 'movie': 0.5274147535813526, 'book': 0.49596100915202107, 'color': 0.5680000716760845}


In [58]:
#Extract a number of favorite noun's for each topic for easy access
topic_favorites = {}
select_n = 5

for topic in memory_topics:
    topic_list = like_memory.loc[like_memory['topic'] == topic]
    topic_list = topic_list.sort_values(by=['sentiment'], ascending=False)
    temp_l = []
    for i in range(select_n):
        temp_l.append(topic_list.subject.iloc[i])#,topic_list.sentiment.iloc[i]))
    topic_favorites[topic] = temp_l
  
print(topic_favorites)
#memory_topics = topic_favorites.keys()

{'animal': ['tiger', 'sheep', 'bear', 'pig', 'horse'], 'food': ['eggs', 'rice', 'noodles', 'gyro', 'pasta'], 'hobby': ['scuba divigin', 'music', 'gardening', 'woodworking', 'karaoke'], 'TV show': ['The Sopranos', 'Fargo', 'The Wire', 'Stranger Things', 'Friends'], 'fruit': ['banana', 'grape', 'cherry', 'tangerine', 'kiwi'], 'music': ['rap', 'dance music', 'metal', 'classical', 'hip-hop'], 'dessert': ['cannoli', 'fortune cookie', 'rice pudding', 'popover', 'Neapolitan ice cream'], 'sport': ['baseball', 'badminton', 'Tennis', 'hockey', 'skiing'], 'genre': ['comedy', 'crime', 'action', 'historical fiction', 'romance'], 'interest': ['to play board games', 'to sing', 'to watch sports', 'to watch tv shows', 'to play sports'], 'movie': ['Joker', 'Pulp fiction', 'Goodfellas', 'The Shining', 'The Shawshank Redemption'], 'book': ['To Kill a Mockingbird', 'Lord of the Flies', 'The Da Vinci Code', 'The Fault in our Stars', 'The Hunger Games'], 'color': ['yellow', 'purple', 'green', 'teal', 'red']}

In [59]:
topic_dislike = {}
for topic in memory_topics:
    topic_list = like_memory.loc[like_memory['topic'] == topic]
    topic_list = topic_list.sort_values(by=['sentiment'])
    temp_l = []
    for i in range(select_n):
        temp_l.append(topic_list.subject.iloc[i])#,topic_list.sentiment.iloc[i]))
    topic_dislike[topic] = temp_l
  
print(topic_dislike)

{'animal': ['beaver', 'ferret', 'duck', 'lion', 'dolphin'], 'food': ['steak', 'pizza', 'hamburger', 'french fries', 'ratatouille'], 'hobby': ['pottery', 'art', 'playing darts', 'sewing', 'karaoke'], 'TV show': ['Hunters', 'Better Call Saul', 'Game of Thrones', 'The Walking Dead', 'The Mandalorian'], 'fruit': ['coconut', 'citrus', 'citron', 'papaya', 'blueberry'], 'music': ['reggae', 'jazz', 'rock', 'EDM', 'house'], 'dessert': ['cookie', 'strudel', 'chocolate bar', 'Danish pastry', 'vanilla cream pie'], 'sport': ['golf', 'american football', 'Soccer', 'volleyball', 'boxing'], 'genre': ['science fiction', 'fantasy', 'mystery', 'historical', 'horror'], 'interest': ['to draw', 'to listen to music', 'to read books', 'to play games', 'to play sports'], 'movie': ['Frozen', '1917', 'The Dark Knight', 'Avengers', 'Parasite'], 'book': ['Les Misérables', 'Pride and Prejudice', 'Gone with the Wind', 'The Chronicles of Narnia', 'Twilight'], 'color': ['turquoise', 'black', 'blue', 'white', 'grey']}


In [60]:
#Default values
sentiment_opt_pos = ["like", "likes", "love", "loves"]
sentiment_opt_neg = ["dislikes", "dislike", "hate", "hates"]
sentiment_opt = sentiment_opt_pos + sentiment_opt_neg
wildcards = {"noun": '<noun>', "sentiment":'<sentiment>', "topic" : "<topic>",
             "agent_sentiment" : '<sentiment_1>', "noun_1" : '<noun_1>', "noun_2" : '<noun_2>',
             "noun_3" : '<noun_3>'}
question_sentiment = "like" #default sentiment is to ask if you like something


In [61]:
def fetch_subject_sentiment(key):
    key = key.lower()
    ans_sent = None

    temp_l = like_memory.loc[like_memory['subject'] == key]

    if len(temp_l) > 0:
        ans_sent = temp_l.sentiment.iloc[0]
        #print(temp_l) #if subject listed under multiple 
    #print(key, ans_sent)
    return ans_sent

In [62]:
#Input subject to find topic: Apple -> Food/Fruit
def fetch_noun_relations(noun):
    temp_noun_set = set()
    query_noun = noun
    api_path = 'http://api.conceptnet.io/query?start=/c/en/' + query_noun + '&rel=/r/IsA'
    obj = requests.get(api_path).json()
    #print(obj['edges'][0])
    
    #outer for traverses the edges, inner for traverses the content in the 'end' tag
    for j in range(len(obj['edges'])):
        #print("Description:", obj['edges'][j]['surfaceText'])
        for i in obj['edges'][j]['end']:
            #if i == 'label':
             #   print("Label:", obj['edges'][j]['end'][i]) 
            #elif i == 'sense_label':
             #   print("Sense_label:", obj['edges'][j]['end'][i])
            temp_string = obj['edges'][j]['end'][i]
            text = nlp(temp_string)
              
            for token in text:
                tag = nltk.pos_tag([token.text])
                if token.pos_ == "NOUN" or tag == "NN" or tag == 'NNS':
                    temp_noun_set.add(token.text)
       
    #print(temp_noun_set)
    return temp_noun_set

In [63]:
#input noun-> find noun's IsA relations e.g Apple is a fruit -> compare IsA relations with existing topics.
def check_noun_topic_exist_memory(noun):
    temp_noun_set = fetch_noun_relations(noun)
    
    union_topics = []
    for topic in memory_topics:
        for item in temp_noun_set:
            if item == topic:
                union_topics.append(item)
    return union_topics

In [64]:
#Check if noun is a known subject in memory
def is_noun_existing_subject(noun):
    temp_l = like_memory.loc[like_memory['subject'] == noun]
    if len(temp_l) > 0:
        return True
    else:
        return False

In [65]:
#Check if noun is a known topic in memory
def is_noun_existing_topic(noun):
    return noun in memory_topics

In [66]:
def similarity_calc(X,Y):
    X = X.lower() #input(q).lower() 
    Y = Y.lower() #input(form_input).lower() 

    # tokenization 
    X_list = word_tokenize(X)  
    Y_list = word_tokenize(Y) 

    # sw contains the list of stopwords 
    sw = stopwords.words('english')  
    l1 =[];l2 =[] 

    # remove stop words from string 
    X_set = {w for w in X_list}# if not w in sw}  
    Y_set = {w for w in Y_list} #if not w in sw} 

    # form a set containing keywords of both strings  
    rvector = X_set.union(Y_set)  
    for w in rvector: 
        if w in X_set: l1.append(1) # create a vector 
        else: l1.append(0) 
        if w in Y_set: l2.append(1) 
        else: l2.append(0) 
    c = 0

    # cosine formula  
    for i in range(len(rvector)): 
            c+= l1[i]*l2[i] 
    cosine = c / float((sum(l1)*sum(l2))**0.5)
    return cosine

In [67]:
#process the user input 
def process_user_input(user_input):
    extracted_nouns = []
    form_input = user_input
    global question_sentiment
    question_sentiment = "like"
    global like_memory
    noun = None
    sentiment_exist = False
    noun_topics = []
    text = nlp(user_input)
    
    for token in text:
        if token.text in sentiment_opt:
            question_sentiment = token.text
            sentiment_exist = True
            
        tag = nltk.pos_tag([token.text])
        if token.pos_ == "NOUN" or tag[0][1] == "NN" or tag[0][1] == 'NNS':
            extracted_nouns.append(token.lemma_)
            extracted_nouns.append(token.text) 
    for n in extracted_nouns:
        if is_noun_existing_topic(n):
            noun_topics = [noun]
            noun = n
            break
            
        noun_topics = check_noun_topic_exist_memory(n)
        if is_noun_existing_subject(n):
            noun = n
            break
        elif len(noun_topics) >0:
            noun = n
            #if the noun is not a known subject (Apple, Soccer, Pasta) then add it with random sentiment  
            noun_sent = np.random.random(1)
            for topic in noun_topics:
                like_memory = like_memory.append({'subject' : noun , 'topic' : topic, 'sentiment' : noun_sent} , ignore_index=True)
            break
    
    if noun != None:
        if is_noun_existing_topic(noun):
            noun_topics = [noun]
            form_input = form_input.replace(noun, wildcards["topic"])
        else:
            form_input = form_input.replace(noun, wildcards["noun"])
    if sentiment_exist:
        form_input = form_input.replace(question_sentiment, wildcards['sentiment'])

    #print(user_input, form_input, noun)
    return form_input, noun, noun_topics, user_input, extracted_nouns, 

In [68]:
def find_question_n_answer_retrieval(user_input):
    max_sim = 0
    max_sim_q = None
    answer_id = 0
   
    for i in range(len(retrieval_q)):
        q = retrieval_q.question[i]
        qid = retrieval_q.answer_id[i]
        
        cosine = similarity_calc(q,user_input)

        if int(cosine) == 1:
            max_sim_q = q
            answer_id = qid
            max_sim = cosine
            break
        elif cosine > max_sim:
            max_sim = cosine
            max_sim_q = q
            answer_id = qid
            
    fetch_answer = retrieval_a.loc[retrieval_a['answer_id'] == answer_id]
    answer = fetch_answer.sample().iloc[0].answer
    #print(max_sim_q, max_sim)
    return answer, answer_id, max_sim_q, max_sim
    
    

In [69]:
#find a suitable question template and return it
def find_question_template(processed_text_input):
    max_sim = 0
    max_sim_q = None
    answer_id = 0
    sent_is_positive = False
    if question_sentiment in sentiment_opt_pos:
        sent_is_positive = True
    for i in range(len(template_q)):
        if sent_is_positive == False and template_q.default_positive[i] == 1:
            continue
        q = template_q.question[i]
        qid = template_q.answer_id[i]
        
        cosine = similarity_calc(q,processed_text_input)

        if int(cosine) == 1:
            max_sim_q = q
            answer_id = qid
            max_sim = cosine
            break
        elif cosine > max_sim:
            max_sim = cosine
            max_sim_q = q
            answer_id = qid
        
    #print(max_sim_q, max_sim)
    return answer_id, max_sim_q, max_sim

In [70]:
def fetch_answer_template(answer_id, noun, noun_topics):
    global like_memory
    global question_sentiment
    fetch_answer = template_a.loc[template_a['answer_id'] == answer_id]
    #default
    ans_sentiment = "like"
    ans_sent_val = None
    ret_nouns = noun 
    for key in memory_topics:
        if noun == key:
            if question_sentiment in sentiment_opt_pos:
                ret_nouns = topic_favorites[key]
                ans_sent_val = 0.5
                break
            else:
                ret_nouns = topic_dislike[key]
                ans_sent_val = 0.5
                #question_sentiment = "dislike"
                if answer_id != 1:
                    ans_sentiment = "hate"
                break
    #if the noun is not a topic (Food/Sports/...)
    if ans_sent_val == None:
        ans_sent_val = fetch_subject_sentiment(noun)
        ans_sentiment = sent_float_to_text(ans_sent_val)

    #the user is talking about something we don't handle in memory.
    elif ans_sent_val == None and noun_topics == None:
        #todo
        pass

        
            
    if (((ans_sentiment in sentiment_opt_pos) and (question_sentiment in sentiment_opt_pos)) 
        or ((ans_sentiment in sentiment_opt_neg) and (question_sentiment in sentiment_opt_neg))):
        fetch_answer = fetch_answer.loc[fetch_answer['same_sentiment'] == 1]
    else:
        fetch_answer = fetch_answer.loc[fetch_answer['same_sentiment'] == 0]
        
    #fetch_answer = template_a.loc[template_a['answer_id'] == answer_id ]
    return fetch_answer, ret_nouns, ans_sentiment

In [71]:
def sent_float_to_text(sentiment):
    ret_sentiment = "love"
    if sentiment < 0.1:
        ret_sentiment = "hate"
    elif sentiment < 0.5:
        ret_sentiment = "dislike"
    elif sentiment < 0.9:
        ret_sentiment = "like"
        
    return ret_sentiment
        

In [72]:
def process_agent_output(answer_template, noun, nouns, noun_topics, answer_sentiment):
    agent_output = answer_template.answer
    temp_nouns = nouns
    #print(agent_output, nouns, noun_topics, (nouns))
    if answer_template.fetch_count > 0 and noun_topics != None and len(noun_topics) >0:
        #print(noun_topics)
        if question_sentiment in sentiment_opt_pos:
            temp_nouns = topic_favorites[noun_topics[0]]
        elif question_sentiment in sentiment_opt_neg:
            temp_nouns = topic_dislike[noun_topics[0]]
            
    #replace nouns
    for i in range(1,answer_template.fetch_count+1):
        temp = "noun_"+str(i)
        agent_output = agent_output.replace(wildcards[temp], temp_nouns[i-1])
    
    if answer_template.use_noun:
        agent_output = agent_output.replace(wildcards["noun"], noun)
    if answer_template.use_sentiment:
        agent_output = agent_output.replace(wildcards["sentiment"], question_sentiment)
    agent_output = agent_output.replace(wildcards["agent_sentiment"], answer_sentiment)
    #print(agent_output)
    return agent_output
    

# GPT-2 Model code

In [74]:
model_name = '124M'
run_name ="run_10"
#gpt2.download_gpt2(model_name=model_name)

In [75]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name=run_name)

Loading checkpoint checkpoint\run_10\model-200
INFO:tensorflow:Restoring parameters from checkpoint\run_10\model-200


In [76]:
def generate_reply(user_question, num_answers=1):
    text_input = "<|startoftext|>" + user_question
    gen_ans =gpt2.generate(sess,
                  run_name=run_name,
                  length=40,
                  temperature=1,
                  prefix=text_input,
                  truncate="<|endoftext|>",
                  include_prefix=False,
                  nsamples=num_answers,
                  batch_size=num_answers,
                  top_p=0.9,
                  return_as_list=True
                  )
    return gen_ans

# Interact with agent

In [87]:
input_sentence = ''
while(1):
    try:
        # Get input sentence
        input_sentence = input('> ')
        # Check if it is quit case
        if input_sentence == 'q' or input_sentence == 'quit': break

        processed_text_input, user_noun, noun_topics  = process_user_input(input_sentence)[0:3]
        answer_id = find_question_template(processed_text_input)[0]
        #print(answer_id)
        if user_noun != None:

            answer, nouns, answer_sentiment = fetch_answer_template(answer_id, user_noun, noun_topics)
            #print('\n',input_sentence)#, question_sentiment)
            print(process_agent_output(answer.sample().iloc[0], user_noun, nouns,noun_topics, answer_sentiment))
        else:
            ans = generate_reply(input_sentence)
            #print('\n',input_sentence)#, question_sentiment)
            print(ans[0])
    except KeyError:
        print("Error: Encountered unknown word.")

> What do you like?

 What do you like?
 I like travel.
> What more do you like ?

 What more do you like ?
 I like travel.
> Where do you like to travel?

 Where do you like to travel?
 I have been to a few countries in Europe which I really enjoyed, hoping to go back soon.
> What tv shows do you like?

 What tv shows do you like?
 NCIS, Modern Family.
> Do you like apples?

 Do you like apples?
No, I don't like apple but I like eggs.
> What fruits do you like?

 What fruits do you like?
I like banana.
> Do you like any other fruits?

 Do you like any other fruits?
I like banana the most! But I also like grape.
> What food do you like?

 What food do you like?
I like eggs and rice.
> What do you think about rice?

 What do you think about rice?
I like it!
> What food do you dislike

 What food do you dislike
I dislike steak and pizza.
> what do you think about pizza?

 what do you think about pizza?
Hm...I don't really like pizza that much.
> Why do you dislike pizza?

 Why do you dis

# Trash below

In [264]:
#fetch_subject_sentiment("berry")
#print(fetch_noun_relations("candy"))
#print(check_noun_topic_exist_memory("sugar") )

user_input = "Do you have any movies that you hate?"
processed_text_input, user_noun, noun_topics  = process_user_input(user_input)[0:3]
#print(processed_text_input, user_noun, noun_topics)
answer_id = find_question_template(processed_text_input)[0]
#print(answer_id)
if user_noun != None:
    
    answer, nouns, answer_sentiment = fetch_answer_template(answer_id, user_noun, noun_topics)
    print('\n',user_input)#, question_sentiment)
    print(process_agent_output(answer.sample().iloc[0], user_noun, nouns,noun_topics, answer_sentiment))


Do you have any movies that you hate? Do you have any <topic>s that you <sentiment>? movie
do you <sentiment> any type of <topic>? 0.7627700713964739

 Do you have any movies that you hate?
['movie']
I hate Joker the most! But I also hate Frozen.


In [86]:
generate_reply("What do you like to do on the weekend?")

[' I like to go to a movie night and not worry about getting in the door. I like to go running around.']

In [121]:
fetch_noun_relations("nourishment")
text = nlp("I likes bananas, do you like to eat banana?")
for token in text:
    print(token.text, token.lemma_)
    #if token.pos_ == "NOUN":
        #temp_noun_set.add(token.text)

I -PRON-
likes like
bananas banana
, ,
do do
you -PRON-
like like
to to
eat eat
banana banana
? ?


In [147]:
df = pd.DataFrame()
for topic in memory_topics:
    topic_list = like_memory.loc[like_memory['topic'] == topic]
    
    divisor = len(topic_list)
    for i in range(divisor):
        noun = topic_list.subject.iloc[i]
        noun_topics = check_noun_topic_exist_memory(noun)
        
        if len(noun_topics) >0:
            #if the noun is not a known subject (Apple, Soccer, Pasta) then add it with random sentiment  
            noun_sent = np.random.random(1)
            for topic in noun_topics:
                df = df.append({'sentiment' : noun_sent, 'topic' : topic, 'subject' : noun } , ignore_index=True)
            


In [149]:
for topic in memory_topics:
    topic_list = df.loc[like_memory['topic'] == topic]
    
    divisor = len(topic_list)
    for i in range(divisor):
        noun = topic_list.subject.iloc[i]
        print(noun)

almond
almond
anchovy
anise
appetizer
apple
apple
apricot
apricot
artichoke
asparagus
aspic
avocado
avocado
bacon
bagel
banana
banana
barbecue
barley
basil
batter
beef
beet
berry
berry
biscuit
bitter
blackberry
blackberry
blueberry
blueberry
bowl
boysenberry
bran
bread
breadfruit
breadfruit
breakfast
brisket
broccoli
brownie
brunch
buckwheat
burrito
butter
cake
cake
candy
caramel
carrot
cashew
casserole
cauliflower
caviar
celery
cereal
chard
cheddar
cheese
cheesecake
chef
cherry
cherry
chew
chicken
chili
chips
chives
chocolate
chow
chutney
cinnamon
citron
citrus
citrus
clam
cobbler
coconut
coconut
cod
coffee
coleslaw
cook
cookie
corn
cornmeal
crab
cranberry
cranberry
cream
cucumber
cucumber
cuisine
cupcake
curry
custard
dairy
dessert
diet
dill
dinner
dip
dish
dough
doughnut
dressing
drink
durian
durian
egg
eggplant
eggplant
elderberry
elderberry
entree
fat
feed
fennel
fig
fig
fillet
fish
flan
flour
fritter
frosting
fruit
garlic
gastronomy
gelatin
ginger
gingerbread
grain
granola
grape


In [151]:
df.to_csv(r'data/memory_structure_copy.csv', index = False)

In [53]:
print(find_question_n_answer_retrieval("do you like to listen to music?"))

('I like a lot of genres, although I have been really into contemporary classical.', 5.0, 'What kind of music do you like to listen to?', 0.8366600265340756)
